In [10]:
import sys
sys.path.append('/home/marc/dev/')

from plotart.streamlines.streamlinesgpt import StreamLines
from plotart.rectangles.rect import RectanglePacker
from helper import create_random_config
import matplotlib.pyplot as plt
from shapely.geometry import MultiLineString



In [11]:
def generate() -> MultiLineString:
    """
    A python module called by the `script` command must implement this function, which takes
    no arguments and return a MultiLineString object.
    :return: resulting MultiLineString
    """
    rect_target = {"x": 0, "y": 0, "h": 30, "w": 42}
    packer = RectanglePacker(rect_target)

    rect_sources = packer.generate_random_rectangles(50, [3, 10], [3, 10])
    solution = packer.optimize_rectangles(rect_sources)
    # packer.plot_rectangles(solution)
    ms = []
    for rect in tq.tqdm(solution):
        #print(f"processing {rect}")
        grid = {
            "x_start": 0,
            "x_end": solution[rect]["w"] * 10,
            "y_start": 0,
            "y_end": solution[rect]["h"] * 10,
            "no_points_x": 100,  # solution[rect]["w"] * 10,
            "no_points_y": 100,  # solution[rect]["h"] * 10,
        }
        sl = streamLines(grid)
        while True:
            try:
                config = create_random_config(
                    grid=grid,
                    n_streamtraces=-1,
                    max_iter=-1,
                    seed_setting=[],
                    n_source_sink=-1,
                )

                for source_sink in config["source_sink"]:
                    sl.add_source_sink(*source_sink)

                for vortex in config["vortices"]:
                    sl.add_vortex(*vortex)
                dt = (solution[rect]["w"] * solution[rect]["h"]) / 1e3
                #print(f"dt = {dt}")
                sl.calc_streamtraces(
                    n_streamtraces=config["n_streamtraces"],
                    seeds=config["seed_setting"],
                    dt=dt,
                    maxiter=config["max_iter"],
                    radius=config["seed_radius"],
                    max_sink_dist=config["max_sink_dist"],
                    n_cpu=4,
                )

                for z in sl.streamtraces:
                    if len(z) > 2:
                        border = -0.1 + 0.2 * np.random.rand()
                        streamtrace = [
                            (
                                x * (1 - border) + solution[rect]["x"] * 10 + border,
                                y * (1 - border) + solution[rect]["y"] * 10 + border,
                            )
                            for x, y in z
                        ]
                        ms.append(streamtrace)

            except Exception as ex:
                continue
            else:
                # print(str(config) + ';')
                if True:
                    for streamline in sl.streamtraces:
                        plt.plot([x[0] for x in streamline], [x[1] for x in streamline])
                    plt.axis("equal")
                    plt.show()
                    c = input("like what you see?")
                    if c not in ["y", "Y", "j", "J"]:
                        continue
                else:
                    break

    return MultiLineString(ms)


if __name__ == "__main__":
    mls = generate()

    for ls in mls:
        plt.plot(*ls.xy, "-", color="k", linewidth=0.1)

    plt.axis("equal")
    plt.show()


NameError: name 'tq' is not defined